In [1]:
from IPython.display import IFrame

IFrame(src='nyt.png', width=1200, height=550)

In [2]:
import pandas as pd
import altair as alt

In [3]:
df = pd.read_csv('GLB.Ts_dSST.csv', skiprows=[0])
df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,-0.29,-0.18,-0.11,-0.19,-0.11,-0.23,-0.21,-0.09,-0.16,-0.23,-0.20,-0.23,-0.19,***,***,-0.14,-0.18,-0.20
1,1881,-0.15,-0.17,0.04,0.04,0.02,-0.20,-0.06,-0.02,-0.13,-0.20,-0.21,-0.10,-0.10,-.11,-.18,0.03,-0.09,-0.18
2,1882,0.15,0.15,0.04,-0.18,-0.16,-0.26,-0.20,-0.05,-0.10,-0.24,-0.16,-0.24,-0.10,-.09,.06,-0.10,-0.17,-0.17
3,1883,-0.31,-0.39,-0.13,-0.17,-0.20,-0.12,-0.08,-0.15,-0.20,-0.14,-0.22,-0.16,-0.19,-.20,-.31,-0.16,-0.12,-0.19
4,1884,-0.15,-0.08,-0.37,-0.42,-0.36,-0.40,-0.34,-0.26,-0.27,-0.24,-0.30,-0.28,-0.29,-.28,-.13,-0.39,-0.34,-0.27


In [4]:
# I didn`t understand why we shoul use wide_to_long.
# My only quess was to transform the months and years to one continuous value, the code is here, but I didn`t use it:


# df.drop(['J-D', 'D-N', 'DJF', 'MAM', 'JJA', 'SON'], axis = 1, inplace = True)
# df.columns = ['Year', 'm1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12']
# df = pd.wide_to_long(df, stubnames='m', i='Year', j='month')
# df = df.reset_index(level=[0,1])
# df["new_year"] = df["Year"] + df["month"]/12 
# df = df.sort_values(by=['new_year'])
# df.drop(['Year', 'month'], axis = 1, inplace = True)
# df.columns = ['temperature', "year"]
# df = df[["year", "temperature"]]
# df.head()
# df[df['year']<1881]

In [5]:
df.drop(['Jan','Feb', 'Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec', 'D-N', 'DJF', 'MAM', 'JJA', 'SON'], axis = 1, inplace = True)

In [6]:
df.columns = ["year", 'temperature']
df.head()

,year,temperature
0,1880,-0.19
1,1881,-0.10
2,1882,-0.10
3,1883,-0.19
4,1884,-0.29


In [7]:
avg1880_1899 = df[(df['year'] >= 1880)&(df['year'] <= 1899)]['temperature'].mean()

avg1880_1899

-0.22849999999999998

In [8]:
df['relational_temperature'] = df['temperature'] - avg1880_1899

In [9]:
chart = alt.Chart(df, 
                  title={"text": ['Annual Global Surface Temperature', 'Relative to Late 19th Century Average'],
    }).mark_point(color='black',
    fill='#f26c08', 
    strokeWidth=1,
    size=55).encode(
        x = alt.X(field='year', 
              title = None,
              type = 'quantitative',
              scale = alt.Scale(
                  zero=False,
                  domain=[1880, 2017]),
              axis = alt.Axis(format='d',
                            values=[1880 + i*10 for i in range(14)],
                            labelOpacity=0.5,
                            labelFontSize=13)),
        y = alt.Y(field = 'relational_temperature', 
              title = None,
              type = 'quantitative',
              axis = alt.Axis(values=[round(-0.4 + i/5, 1) for i in range(9)], 
                            labelExpr="datum.value >=0 ? '+'+datum.value+'\u00b0': datum.value+'\u00b0'",
                            labelOpacity=0.5,
                             labelFontSize=13)
             )
) 

In [10]:
line = alt.Chart(pd.DataFrame({'x': [1880, 2020], 'y': [0, 0]})).mark_line(color="#a9a9a9").encode(
    x = 'x',
    y = 'y',  
) 

In [11]:
text1 = alt.Chart({'values':[{'x': 2020, 'y': 0.12}]}).mark_text(
    text='HOTTER THAN THE\n1880-1899 AVERAGE', lineBreak='\n', fontSize=16, align = 'right').encode(
    x='x:Q', y='y:Q'
)

text2 = alt.Chart({'values':[{'x': 2020, 'y': -0.06}]}).mark_text(
    text='COLDER', fontSize=16, align = 'right').encode(
    x='x:Q', y='y:Q'
)

In [12]:
labeled_points_right = alt.Chart(df.query("year in [1944, 2014, 2015, 2017]")).mark_point().encode(
    x='year:Q',
    y='temperature:Q'
).mark_text(
    align='left',
    baseline='middle',
    dx=7,
    fontSize=16, 
).encode(
    x = "year:Q",
    y = "relational_temperature:Q",
    text='year'
)

labeled_points_left = alt.Chart(df.query("year in [1904, 1998]")).mark_point().encode(
    x='year:Q',
    y='temperature:Q'
).mark_text(
    align='left',
    baseline='middle',
    dx=-45,
    fontSize=16, 
).encode(
    x = "year:Q",
    y = "relational_temperature:Q",
    text='year'
)

labeled_point_max = alt.Chart(df.query("relational_temperature == relational_temperature.max()")).mark_point().encode(
    x='year:Q',
    y='temperature:Q'
).mark_text(
    align='left',
    baseline='middle',
    dx=7,
    fontSize=16,
    fontWeight='bold'
).encode(
    x = "year:Q",
    y = "relational_temperature:Q",
    text='year'
)

In [13]:
(line + text1 + text2 + chart + labeled_points_right + labeled_points_left + labeled_point_max
).properties(width = 957, 
             height = 604,
             background = '#ffffff',
             padding = 25
            ).configure_view(strokeWidth = 0, 
                             strokeOpacity=0
                            ).configure_title(
                                anchor='start',
                                dx = 40,
                                dy = 100,
                                fontSize=16,
                                align = 'left'
                                ).configure_axis(disable = True,
                                                  ticks = False,
                                                  domain = False,
                                                  gridColor='#f3f3f3',
                                                  gridWidth=1)

alt.LayerChart(...)